In [0]:
!pip uninstall s3i
!pip install https://git.rwth-aachen.de/kwh40/s3i/-/jobs/artifacts/master/raw/public/s3i-0.4-py3-none-any.whl?job=wheel

In [0]:
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable varInspector/main

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from s3i import UserMessage,IdentityProvider, TokenType, Broker, Key, Directory
import os
import uuid
import base64

In [0]:
USERNAME = "KWH-Team" #"adelbert" #input('[S3I]: Please enter the username:')
PASSWORD = "V4yuJ31izh0GCH36954O" #"adelbert" #"V4yuJ31izh0GCH36954O" #getpass.getpass('[S3I]: Please enter the password:')

In [0]:
CLIENT = "s3i:602c7401-c381-4c44-9eff-3975d539e271" #"s3i:9b34f12b-f0ab-4f13-b0cc-82202e7a9770" #"s3i:4f076127-d9e2-477d-8d42-4453225bfaaa" #"s3i:3e7cfc9b-2fda-4c56-a4f0-b5e8f34b6da6" #"s3i:e09825e5-a872-4541-9ef4-31ceab014371" #input('[S3I]: Please enter the thing id of this HMI:') #
SECRET = "cd4fc68c-25e8-49ba-a99f-9dac86ce10b7" #"a1f025f3-687c-44ae-8b79-cd7ebdd076c8" #"5ad8387a-5183-471b-8dfe-da2c7c62557f" #"346e9867-e278-4ba3-b336-1421e5abd594" # getpass.getpass('[S3I]: Please enter the secret:') # 

In [0]:
IDP = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)

# Get Token


In [0]:
#idp = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
#                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)
print("[S3I]: Access token: \n ", IDP.get_token(TokenType.ACCESS_TOKEN))

# Add Public Key

In [0]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
key = Key(size=2048)
key.generateKey(USERNAME)
key.exportsecKeyToFile(filename="workshop_sec.asc", path="/content/")
#print(k.key.is_public)
key.exportPubKeyToFile(filename="workshop_pub.asc", path="/content/")

response = dir.putPublicKey("s3i:602c7401-c381-4c44-9eff-3975d539e271","workshop_pub.asc")
print(response.text)

# Send User Messages

## Verify the endpoints

In [0]:
receivers = [str(x) for x in input('[S3I]: Please enter the receivers, seperate with a space:').split()]


In [0]:
#idp = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
#                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
defaultHMIs = [dir.queryThingIDBased(receiver+"/attributes/defaultHMI") for receiver in receivers]
endpoints = [dir.queryThingIDBased(hmi+"/attributes/allEndpoints")[0] for hmi in defaultHMIs]

In [0]:
endpoints = [str(x) for x in input('[S3I]: Please enter the receivers, seperate with a space:').split()]

## Prepare the message

In [0]:
subject = input('[S3I]: please enter the subject: \n')
text = input('[S3I]: please enter the text: \n')
msg_uuid = "s3i:" + str(uuid.uuid4())
replyToEndpoint=None
attachments=None

### Add attachement to message

In [0]:
filename_list = list(map(str, input("[S3I]: Please enter the name of file that you want to send: ").split()))
attachments = list()

for filename in filename_list:
    attachment_dict = dict()
    attachment_dict["filename"] = filename
    with open("/content/attachment_files/"+filename, "rb") as f:
        # encode an attachment file to BASE64 bytes
        base64_data = base64.b64encode(f.read())
        data = base64_data.decode()  # convert byte to str
        attachment_dict["data"] = data
        attachments.append(attachment_dict)

## Assemble User Message

In [0]:
message = UserMessage(sender=CLIENT, identifier=msg_uuid,receivers=receivers, subject=subject, text=text,
                  replyToEndpoint=replyToEndpoint, attachments=attachments)
print(message.msg)

## Sign the message

In [0]:
pgpKey_sec = Key(path_demo_dir="/content/", filename="workshop_sec.asc")
message.sign(pgpKey_sec.key)
print(message.pgpMsg.__str__())

## Encrypt the message

In [0]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
publicKeys_str = dir.getPublicKey("s3i:602c7401-c381-4c44-9eff-3975d539e271") #receivers)
publicKeys=[Key(key_str=publicKeys_str)]
#publicKeys = [Key(key_str=i) for i in publicKeys_str]
message.encrypt(publicKeys)
#print(message.pgpMsg.__str__())

## Send the message via S³I Broker

In [0]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
broker = Broker(auth_form='Username/Password', username=" ", password=access_token, host="rabbitmq.s3i.vswf.dev")
broker.send(endpoints, message.pgpMsg.__str__())